# Galaxy Life Planner Demo

This notebook demonstrates the Galaxy Life Planner system, now using a Pohang daily routine with local POIs as the working example.


In [50]:
# Change to the src directory to access the glp_demo module
import os
import sys

# Add src directory to Python path
src_path = os.path.join(os.getcwd(), 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

print(f"Added {src_path} to Python path")
print(f"Current working directory: {os.getcwd()}")

Added c:\Users\7info\Desktop\삼성 과제\GLP_demo\src to Python path
Current working directory: c:\Users\7info\Desktop\삼성 과제\GLP_demo


In [51]:
# Import and run the GLP demo
from glp_demo import demo

# Run the demo
demo.main()

Instruction Planner Order -> fetch_routine_model, query_personal_graph, query_common_graph, rank_candidates
--- Recommendations ---
Fit365 Gym: score=0.646
  reasoning={'routine': 0.0, 'personal': 1.0, 'intent': 1.0, 'distance': 0.9629350244519002}
  message=How about Fit365 Gym (gym)? aligns with recent visits / fits the current app signals / close to your current location.

Hanok Kitchen: score=0.539
  reasoning={'routine': 0.0, 'personal': 1.0, 'intent': 0.6274509803921569, 'distance': 0.8236466356484318}
  message=How about Hanok Kitchen (korean_restaurant)? aligns with recent visits / fits the current app signals / close to your current location.



## Understanding the Results

The demo shows:
1. **Instruction Planner Order**: The sequence of operations the system performs
2. **Recommendations**: Scored candidates with detailed reasoning
3. **Natural Language Explanations**: Human-readable descriptions of why each place was recommended

## Zero-shot LLM Scoring (Optional)

Use the `ZeroShotLLMScorer` to blend an open-source text generation model into the recommendation engine.
The snippet below defaults to a lightweight chat checkpoint; switch to any model available in your local Hugging Face cache.

- Explanations now come in English with a concise reason summary.
- Install `transformers`, `accelerate`, and download the model locally; this cell requests GPU execution (`device_map="cuda"`).


In [ ]:
# Optional: integrate an open-source LLM into zero-shot scoring
from datetime import datetime

import importlib

from glp_demo.app_signals import infer_intent_signals
from glp_demo.ckg import CommonKnowledgeGraph
from glp_demo.demo_data import create_sample_events, create_sample_pois
from glp_demo.pkg import PersonalKnowledgeGraph
from glp_demo.process_mining import build_routine_model
import glp_demo.llm as llm_module
import glp_demo.recommendation as rec_module

# Reload to pick up the latest implementations if modules were imported earlier
ZeroShotLLMScorer = getattr(importlib.reload(llm_module), "ZeroShotLLMScorer")
ExplanationGenerator = getattr(importlib.reload(llm_module), "ExplanationGenerator")
RecommendationEngine = getattr(importlib.reload(rec_module), "RecommendationEngine")

events = create_sample_events()
routine = build_routine_model(events)
pkg = PersonalKnowledgeGraph()
pkg.ingest_events(events)
pkg.link_routine(routine)
ckg = CommonKnowledgeGraph(create_sample_pois())
reference_time = datetime(2025, 9, 26, 18, 30)
intents = infer_intent_signals(events, reference_time)

try:
    scorer = ZeroShotLLMScorer(
        model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
        device_map="cuda",
    )
except RuntimeError as exc:
    print(f"[WARN] zero-shot scorer disabled -> {exc}")
    scorer = None

try:
    explanation_generator = ExplanationGenerator(
        device_map="cuda",
    )
except RuntimeError as exc:
    print(f"[WARN] explanation LLM disabled -> {exc}")
    explanation_generator = None

engine = RecommendationEngine(routine, pkg, ckg, zero_shot_scorer=scorer)
recommendations = engine.recommend(
    latitude=36.0539,
    longitude=129.3745,
    reference_time=reference_time,
    intents=intents,
    radius_km=3.0,
    limit=3,
)

for item in recommendations:
    poi = item.candidate.poi
    if explanation_generator is not None:
        message = explanation_generator.build_message(item)
    else:
        message = f"Fallback explanation -> {item.candidate.reasoning_tokens}"
    print(f"{poi.name} -> score={item.candidate.score:.3f}, reasoning={item.candidate.reasoning_tokens}")
    print(f"  message={message}\n")

## Zero-Shot Recommendation Demo

This section demonstrates how the system handles **zero-shot POIs** (places the user has never visited before).

### Key Features:
- **Context-aware scoring**: LLM analyzes user intent, time, and location
- **Semantic understanding**: Matches POI categories with user intentions
- **Exploration boost**: Encourages discovery of new relevant places

### Example Scenario:
User has strong exercise intent (fitness app usage) but encounters a new gym they have never visited.

In [ ]:
# Zero-Shot Recommendation with Mock LLM Scorer
from datetime import datetime
import importlib

from glp_demo.demo_data import create_sample_events, create_sample_pois
from glp_demo.pkg import PersonalKnowledgeGraph
from glp_demo.ckg import CommonKnowledgeGraph
from glp_demo.process_mining import build_routine_model
from glp_demo.app_signals import infer_intent_signals
import glp_demo.recommendation as rec_module

# Reload to get the latest POI data (including new Premium Fitness Club)
RecommendationEngine = getattr(importlib.reload(rec_module), "RecommendationEngine")

# Mock Zero-Shot LLM Scorer
class MockZeroShotScorer:
    """Simulates LLM-based scoring for zero-shot POIs based on context understanding"""
    
    def score(self, context, poi_features):
        scores = {}
        intent_scores = context.get('intent_scores', {})
        time_slot = context.get('time_slot', 'unknown')
        
        for feature in poi_features:
            poi_id = feature['poi_id']
            category = feature['category']
            
            # Context-aware scoring logic
            score = 0.1  # base score
            
            # Strong intent-category matching
            if category == 'gym' and intent_scores.get('exercise', 0) > 0.8:
                score = 0.9  # Perfect match: exercise intent + gym
            elif category == 'italian_restaurant' and intent_scores.get('meal', 0) > 0.5:
                score = 0.7  # Good match: meal intent + restaurant
            elif category == 'coffee_shop' and intent_scores.get('social', 0) > 0.4:
                score = 0.6  # Social intent + cafe
            elif category == 'convenience_store':
                score = 0.3  # Always somewhat useful
            
            # Time-based adjustments
            if time_slot == 'evening' and category == 'gym':
                score += 0.1  # Evening gym bonus
            
            scores[poi_id] = min(1.0, score)  # Cap at 1.0
        
        return scores

# Setup data and models
events = create_sample_events()
routine = build_routine_model(events)
pkg = PersonalKnowledgeGraph()
pkg.ingest_events(events)
pkg.link_routine(routine)
ckg = CommonKnowledgeGraph(create_sample_pois())

reference_time = datetime(2025, 9, 26, 18, 30)  # Evening time
intents = infer_intent_signals(events, reference_time)

print("=== Current User Intents ===")
for intent in intents[:3]:  # Show top 3
    print(f"{intent.intent}: {intent.score:.3f}")

# Create recommendation engine with zero-shot scorer
zero_shot_scorer = MockZeroShotScorer()
engine = RecommendationEngine(routine, pkg, ckg, zero_shot_scorer=zero_shot_scorer)

recommendations = engine.recommend(
    latitude=36.0539,
    longitude=129.3745,
    reference_time=reference_time,
    intents=intents,
    radius_km=3.0,
    limit=5,
)

print("
=== Recommendations (Including Zero-Shot) ===")
for i, item in enumerate(recommendations, 1):
    poi = item.candidate.poi
    reasoning = item.candidate.reasoning_tokens
    visit_count = pkg.place_visit_counts.get(poi.poi_id, 0)
    
    # Determine if this is a zero-shot POI
    is_zero_shot = visit_count == 0
    status = "🆕 ZERO-SHOT" if is_zero_shot else f"📍 Visited {visit_count}x"
    
    print(f"{i}. {poi.name} ({status})")
    print(f"   Final Score: {item.candidate.score:.3f}")
    print(f"   Category: {poi.category}")
    
    # Show score breakdown
    print(f"   Score Breakdown:")
    for signal, score in reasoning.items():
        if signal == 'llm':
            print(f"     🤖 LLM Bonus: {score:.3f} (zero-shot only)")
        else:
            print(f"     {signal}: {score:.3f}")
    
    if is_zero_shot:
        print(f"   💡 Why recommended: High {list(intents)[0].intent} intent matches {poi.category}")
    
    print()
